****STDL PROJECT NOTEBOOK****

**Phase 1: predict gene expression levels of one specific gene over given input biopsy images**

Assign GPU device

In [1]:
# the next 2 lines are to allow debugging with CUDA !
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  
print(f'cuda debugging allowed')

cuda debugging allowed


In [2]:
######
import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()

cuda device count: 1
Using device: cuda
device name: GeForce RTX 2080 Ti
torch.cuda.device(0): <torch.cuda.device object at 0x7fe4141d0c10>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



Import project files

In [3]:
# create code to reimport module if i change it
%load_ext autoreload
%autoreload 2

# note: path to project is: /home/roy.rubin/STDLproject/
import loadAndPreProcess
import deepNetworkArchitechture

perform pre processing actions

In [4]:
print("\nHi! welcome to the program :)\n")


Hi! welcome to the program :)



In [5]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
imageFolder = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [6]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
augmentedImageFolder = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [7]:
path_to_mtx_tsv_files_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData"
matrix_dataframe, features_dataframe, barcodes_datafame = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx

----- finished function load_dataframes_from_mtx_and_tsv -----



remove empty genes (i think there are only 3 of these ... need to verify)

In [8]:
%autoreload 2
matrix_dataframe, mapping_between_old_and_new_indices = loadAndPreProcess.cut_empty_genes(matrix_dataframe)  #TODO: uncomment later

cutting all genes (rows) that contain only zeros ...

print data regarding the reduced dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23073 entries, 0 to 23072
Columns: 4992 entries, 0 to 4991
dtypes: Sparse[int64, 0](4992)
memory usage: 245.2 MB
None
   0     1     2     3     4     5     6     7     8     9     ...  4982  \
0     0     0     0     0     0     0     0     0     0     1  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     2     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   4983  4984  4985  4986  4987  4988  4989  4990  4991  
0     0     0     0     0     0     0     0     0     0  
1     0     0     1     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0  

In [9]:
# uncomment if wanted
# print(f'\note: this is the mapping_between_old_and_new_indices: \n{mapping_between_old_and_new_indices}')

In [10]:
gene_name = 'MKI67'
custom_DS_SingleValuePerImg = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)
custom_DS_SingleValuePerImg_augmented = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=augmentedImageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----



In [11]:
k = 10
custom_DS_KGenesWithHighestVariance = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [12]:
k = 10
custom_DS_LatentTensor_NMF = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [13]:
%autoreload 2

k = 10
custom_DS_LatentTensor_AE = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder -----
initializing the autoencoder (this might take a while) ...

----- entered function return_trained_AE_net -----
--delete-- verify inside STDL_Dataset_matrix_df_for_AE_init __init__: self.data.shape: (23073, 4992)
note - number of (hidden) linear layers is supposed to be 3

entered __init__ of AutoencoderNet
****** begin training ******

iteration 1 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.004043815657496452 max loss 1809.9281005859375
               average loss 12.162598668217658

iteration 2 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.003786135232076049 max loss 1557.9254150390625
               average loss 10.252500305712573

iteration 3 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.00356870680116117 max loss 1534.355712890625
               average loss 10.024925


display a few sample images

In [14]:
# TODO: code to display some sample images
# 

check sizes of a pair (x,y) from all different datasets

In [15]:
print(f'custom_DS_SingleValuePerImg (single gene):      x shape {custom_DS_SingleValuePerImg[0][0].shape}, y is an int')
print(f'custom_DS_SingleValuePerImg_augmented:          x shape {custom_DS_SingleValuePerImg_augmented[0][0].shape}, y is an int')
print("---------")
print(f'custom_DS_KGenesWithHighestVariance:            x shape {custom_DS_KGenesWithHighestVariance[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance[0][1].shape}')
print(f'custom_DS_KGenesWithHighestVariance_augmented:  x shape {custom_DS_KGenesWithHighestVariance_augmented[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance_augmented[0][1].shape}')
print("---------")
print(f'custom_DS_LatentTensor_NMF:                     x shape {custom_DS_LatentTensor_NMF[0][0].shape}, y shape {custom_DS_LatentTensor_NMF[0][1].shape}')
print(f'custom_DS_LatentTensor_NMF_augmented:           x shape {custom_DS_LatentTensor_NMF_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_NMF_augmented[0][1].shape}')
print("---------")
print(f'custom_DS_LatentTensor_AE:                      x shape {custom_DS_LatentTensor_AE[0][0].shape}, y shape {custom_DS_LatentTensor_AE[0][1].shape}')
print(f'custom_DS_LatentTensor_AE_augmented:            x shape {custom_DS_LatentTensor_AE_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_AE_augmented[0][1].shape}')


custom_DS_SingleValuePerImg (single gene):      x shape torch.Size([3, 176, 176]), y is an int
custom_DS_SingleValuePerImg_augmented:          x shape torch.Size([3, 176, 176]), y is an int
---------
custom_DS_KGenesWithHighestVariance:            x shape torch.Size([3, 176, 176]), y shape (10,)
custom_DS_KGenesWithHighestVariance_augmented:  x shape torch.Size([3, 176, 176]), y shape (10,)
---------
custom_DS_LatentTensor_NMF:                     x shape torch.Size([3, 176, 176]), y shape (10,)
custom_DS_LatentTensor_NMF_augmented:           x shape torch.Size([3, 176, 176]), y shape (10,)
---------
custom_DS_LatentTensor_AE:                      x shape torch.Size([3, 176, 176]), y shape torch.Size([10])
custom_DS_LatentTensor_AE_augmented:            x shape torch.Size([3, 176, 176]), y shape torch.Size([10])


In [48]:
%autoreload 2
import executionModule

# define hyperparameters for the experiments
hyperparameters = dict()
hyperparameters['batch_size'] = 25
hyperparameters['max_alowed_number_of_batches'] = 99999
hyperparameters['precent_of_dataset_allocated_for_training'] = 0.8
hyperparameters['learning_rate'] = 1e-4
hyperparameters['num_of_epochs'] = 5
hyperparameters['channels'] = [32] 
hyperparameters['num_of_convolution_layers'] = len(hyperparameters['channels'])
hyperparameters['hidden_dims'] = [100]
hyperparameters['num_of_hidden_layers'] = len(hyperparameters['hidden_dims'])
hyperparameters['pool_every'] = 99999

In [49]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_SingleValuePerImg, hyperparameters, device)

/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 5
recieved max_alowed_number_of_batches 99999
--delete-- verify batch_size 25
****** begin training ******

iteration 1 of 5 epochs
batch 122 of 122 batches
finished inner loop.

in this epoch: min loss 0.00017852027667686343 max loss 517.2552490234375
               average loss 4.624471161366616
               number of correct predictions: 2797 / 3050

iteration 2 of 5 epochs
batch 122 of 122 batches
finished inner loop.

in 

In [51]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_SingleValuePerImg_augmented, hyperparameters, device)

/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 5
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 5 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 0.0023725370410829782 max loss 131.12777709960938
               average loss 0.2896606674199336
               number of correct predictions: 12003 / 12200

iteration 2 of 5 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 0.001827

In [52]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_KGenesWithHighestVariance, hyperparameters, device)

TypeError: new() received an invalid combination of arguments - got (tuple, int), but expected one of:
 * (torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mtuple[0m, [31;1mint[0m)
 * (object data, torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mtuple[0m, [31;1mint[0m)


In [ ]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_KGenesWithHighestVariance_augmented, hyperparameters, device)

In [ ]:
###

In [ ]:
# executionModule.runExperiment1_singleGenePrediction(custom_DS_SingleValuePerImg, device)
# executionModule.runExperiment1_singleGenePrediction(custom_DS_SingleValuePerImg_augmented, device)

# executionModule.runExperiment2_allGenePrediction_dimReduction_KHighestVariances(custom_DS_KGenesWithHighestVariance, device)
# executionModule.runExperiment2_allGenePrediction_dimReduction_KHighestVariances(custom_DS_KGenesWithHighestVariance_augmented, device)

# executionModule.runTest3_allGenePrediction_dimReduction_NMF(stdl_ds2, device)
# executionModule.runTest4_allGenePrediction_dimReduction_AutoEncoder(stdl_ds4, device)